# Strategies in Federated Learning

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower Strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.0.0 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("../dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("../dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [8]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)


INFO flwr 2023-05-04 11:34:14,320 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-05-04 11:34:15,934	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-04 11:34:16,492 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 15863128473.0, 'node:172.17.0.2': 1.0, 'CPU': 16.0, 'memory': 31726256948.0, 'accelerator_type:RTX': 1.0, 'GPU': 1.0}
INFO flwr 2023-05-04 11:34:16,493 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-04 11:34:16,493 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-04 11:34:16,493 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-04 11:34:16,493 | server.py:101 | FL starting
DEBUG flwr 2023-05-04 11:34:16,494 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=744014) [Client 4] fit, config: {}
(launch_and_fit pid=744014) Epoch 1: train loss 0.06470978260040283, accuracy 0.24222222222222223
(launch_and_fit pid=744130) [Client 1] fit, config: {}
(launch_and_fit pid=744130) Epoch 1: train loss 0.06438679993152618, accuracy 0.2351111111111111
(launch_and_fit pid=744230) [Client 8] fit, config: {}
(launch_and_fit pid=744230) Epoch 1: train loss 0.06375786662101746, accuracy 0.24622222222222223


KeyboardInterrupt: 

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [6]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-05-02 07:57:12,285 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-05-02 07:57:16,362	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-02 07:57:16,905 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 16.0, 'memory': 75102343783.0, 'object_store_memory': 36472433049.0, 'node:172.17.0.2': 1.0, 'accelerator_type:RTX': 1.0}
INFO flwr 2023-05-02 07:57:16,906 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-02 07:57:16,906 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-02 07:57:16,906 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-02 07:57:16,907 | server.py:101 | FL starting
DEBUG flwr 2023-05-02 07:57:16,907 | server.py:218 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1560097) [Client 1] fit, config: {}
(launch_and_fit pid=1560097) Epoch 1: train loss 0.06533655524253845, accuracy 0.21933333333333332
(launch_and_fit pid=1560159) [Client 6] fit, config: {}
(launch_and_fit pid=1560159) Epoch 1: train loss 0.06568127870559692, accuracy 0.2091111111111111
(launch_and_fit pid=1560251) [Client 3] fit, config: {}


DEBUG flwr 2023-05-02 07:57:23,894 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-05-02 07:57:23,898 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-05-02 07:57:23,900 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1560251) Epoch 1: train loss 0.06523174792528152, accuracy 0.2071111111111111
(launch_and_evaluate pid=1560319) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=1560384) [Client 6] evaluate, config: {}


DEBUG flwr 2023-05-02 07:57:29,029 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-05-02 07:57:29,030 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-05-02 07:57:29,030 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-05-02 07:57:48,067 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-05-02 07:57:48,072 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1561028) Epoch 1: train loss 0.11405691504478455, accuracy 0.1411111111111111
(launch_and_evaluate pid=1561095) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=1561156) [Client 3] evaluate, config: {}


DEBUG flwr 2023-05-02 07:57:53,281 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-05-02 07:57:53,282 | server.py:147 | FL finished in 36.37512519804295
INFO flwr 2023-05-02 07:57:53,282 | app.py:218 | app_fit: losses_distributed [(1, 5.935198771158855), (2, 0.7092998193105062), (3, 0.08116747140884399)]
INFO flwr 2023-05-02 07:57:53,282 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-05-02 07:57:53,283 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-05-02 07:57:53,283 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2023-05-02 07:57:53,283 | app.py:222 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=1561217) [Client 5] evaluate, config: {}


History (loss, distributed):
	round 1: 5.935198771158855
	round 2: 0.7092998193105062
	round 3: 0.08116747140884399

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

이게 뭔소리야;;

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [6]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [9]:
# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

In [10]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

hist = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)


metrics_cen = list(hist.metrics_centralized.keys())
print(metrics_cen)

INFO flwr 2023-05-04 11:34:29,979 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-05-04 11:34:33,886	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-04 11:34:34,429 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'GPU': 1.0, 'memory': 31642664142.0, 'node:172.17.0.2': 1.0, 'object_store_memory': 15821332070.0, 'accelerator_type:RTX': 1.0}
INFO flwr 2023-05-04 11:34:34,429 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-04 11:34:34,430 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-04 11:34:34,430 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-04 11:34:34,688 | server.py:91 | initial parameters (loss, other metrics): 0.07383316326141358, {'accuracy': 0.08}
INFO flwr 2023-05-04 11:34:34,688 | server.py:101 | FL starting
DEBUG flwr 2023-05-04 11:34:34,689 | server.py:218 | fit_round 1: strategy sampled 3 clients (out 

Server-side evaluation loss 0.07383316326141358 / accuracy 0.08
(launch_and_fit pid=745634) [Client 7] fit, config: {}
(launch_and_fit pid=745634) Epoch 1: train loss 0.06384707987308502, accuracy 0.25466666666666665
(launch_and_fit pid=745751) [Client 2] fit, config: {}
(launch_and_fit pid=745751) Epoch 1: train loss 0.06492054462432861, accuracy 0.22777777777777777
(launch_and_fit pid=745899) [Client 1] fit, config: {}


DEBUG flwr 2023-05-04 11:34:42,499 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-05-04 11:34:42,503 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-05-04 11:34:42,568 | server.py:119 | fit progress: (1, 0.06464041471481323, {'accuracy': 0.298}, 7.878992336103693)
DEBUG flwr 2023-05-04 11:34:42,568 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=745899) Epoch 1: train loss 0.06482955068349838, accuracy 0.23377777777777778
Server-side evaluation loss 0.06464041471481323 / accuracy 0.298
(launch_and_evaluate pid=746036) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=746149) [Client 5] evaluate, config: {}


DEBUG flwr 2023-05-04 11:34:48,299 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-05-04 11:34:48,299 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-05-04 11:34:48,300 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_evaluate pid=746850) [Client 5] evaluate, config: {}
(launch_and_fit pid=747024) [Client 8] fit, config: {}
(launch_and_fit pid=747024) Epoch 1: train loss 0.052481744438409805, accuracy 0.39466666666666667
(launch_and_fit pid=747096) [Client 5] fit, config: {}
(launch_and_fit pid=747096) Epoch 1: train loss 0.05353112146258354, accuracy 0.366
(launch_and_fit pid=747216) [Client 9] fit, config: {}


DEBUG flwr 2023-05-04 11:35:09,842 | server.py:232 | fit_round 3 received 3 results and 0 failures
INFO flwr 2023-05-04 11:35:09,910 | server.py:119 | fit progress: (3, 0.05188120603561401, {'accuracy': 0.404}, 35.22159571200609)
DEBUG flwr 2023-05-04 11:35:09,911 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05188120603561401 / accuracy 0.404
(launch_and_fit pid=747216) Epoch 1: train loss 0.053625527769327164, accuracy 0.3748888888888889
(launch_and_evaluate pid=747297) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=747413) [Client 9] evaluate, config: {}


DEBUG flwr 2023-05-04 11:35:15,774 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-05-04 11:35:15,774 | server.py:147 | FL finished in 41.085737239103764
INFO flwr 2023-05-04 11:35:15,775 | app.py:218 | app_fit: losses_distributed [(1, 0.06440572420756023), (2, 0.05476273314158122), (3, 0.05119442224502563)]
INFO flwr 2023-05-04 11:35:15,775 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-05-04 11:35:15,775 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-05-04 11:35:15,775 | app.py:221 | app_fit: losses_centralized [(0, 0.07383316326141358), (1, 0.06464041471481323), (2, 0.05538888573646546), (3, 0.05188120603561401)]
INFO flwr 2023-05-04 11:35:15,776 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.08), (1, 0.298), (2, 0.37), (3, 0.404)]}


(launch_and_evaluate pid=747475) [Client 8] evaluate, config: {}
['accuracy']


## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (trainig, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [7]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [8]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [9]:
from flwr.common import Metrics
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [10]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

histFedAvg = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-05-02 08:05:26,884 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-05-02 08:05:30,912	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-02 08:05:31,453 | app.py:180 | Flower VCE: Ray initialized with resources: {'accelerator_type:RTX': 1.0, 'CPU': 16.0, 'node:172.17.0.2': 1.0, 'object_store_memory': 36463837593.0, 'GPU': 1.0, 'memory': 75082287719.0}
INFO flwr 2023-05-02 08:05:31,454 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-02 08:05:31,454 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-02 08:05:31,454 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-02 08:05:31,682 | server.py:91 | initial parameters (loss, other metrics): 0.07375366973876953, {'accuracy': 0.098}
INFO flwr 2023-05-02 08:05:31,682 | server.py:101 | FL starting
DEBUG flwr 2023-05-02 08:05:31,683 | server.py:218 | fit_round 1: strategy sampled 3 clients (out

Server-side evaluation loss 0.07375366973876953 / accuracy 0.098
(launch_and_fit pid=1587430) [Client 4, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1587430) Epoch 1: train loss 0.06484726816415787, accuracy 0.23
(launch_and_fit pid=1587494) [Client 9, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=1587494) Epoch 1: train loss 0.06577346473932266, accuracy 0.21711111111111112
(launch_and_fit pid=1587583) [Client 5, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-05-02 08:05:38,757 | server.py:232 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-05-02 08:05:38,761 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-05-02 08:05:38,820 | server.py:119 | fit progress: (1, 0.061814931631088255, {'accuracy': 0.302}, 7.137673989054747)
DEBUG flwr 2023-05-02 08:05:38,821 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.061814931631088255 / accuracy 0.302
(launch_and_fit pid=1587583) Epoch 1: train loss 0.06493223458528519, accuracy 0.21933333333333332
(launch_and_evaluate pid=1587654) [Client 7] evaluate, config: {}
(launch_and_evaluate pid=1587715) [Client 4] evaluate, config: {}


DEBUG flwr 2023-05-02 08:05:43,921 | server.py:182 | evaluate_round 1 received 3 results and 0 failures
DEBUG flwr 2023-05-02 08:05:43,921 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_evaluate pid=1588169) [Client 3] evaluate, config: {}
(launch_and_fit pid=1588232) [Client 7, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1588232) Epoch 1: train loss 0.04963402450084686, accuracy 0.4242222222222222
(launch_and_fit pid=1588232) Epoch 2: train loss 0.04628428816795349, accuracy 0.4562222222222222
(launch_and_fit pid=1588300) [Client 4, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1588300) Epoch 1: train loss 0.04971098154783249, accuracy 0.4226666666666667
(launch_and_fit pid=1588300) Epoch 2: train loss 0.04688555747270584, accuracy 0.4562222222222222
(launch_and_fit pid=1588364) [Client 8, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}
(launch_and_fit pid=1588364) Epoch 1: train loss 0.05027785524725914, accuracy 0.4137777777777778


DEBUG flwr 2023-05-02 08:06:06,974 | server.py:232 | fit_round 3 received 3 results and 0 failures
INFO flwr 2023-05-02 08:06:07,038 | server.py:119 | fit progress: (3, 0.049360291004180906, {'accuracy': 0.432}, 35.355143822031096)
DEBUG flwr 2023-05-02 08:06:07,038 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=1588364) Epoch 2: train loss 0.04703766852617264, accuracy 0.4573333333333333
Server-side evaluation loss 0.049360291004180906 / accuracy 0.432
(launch_and_evaluate pid=1588437) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=1588498) [Client 5] evaluate, config: {}


DEBUG flwr 2023-05-02 08:06:12,315 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-05-02 08:06:12,315 | server.py:147 | FL finished in 40.63267382001504
INFO flwr 2023-05-02 08:06:12,316 | app.py:218 | app_fit: losses_distributed [(1, 0.06197010064125061), (2, 0.05191275413831075), (3, 0.04704053934415181)]
INFO flwr 2023-05-02 08:06:12,316 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-05-02 08:06:12,316 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.3073333333333333), (2, 0.4046666666666667), (3, 0.47266666666666673)]}
INFO flwr 2023-05-02 08:06:12,316 | app.py:221 | app_fit: losses_centralized [(0, 0.07375366973876953), (1, 0.061814931631088255), (2, 0.05233015179634094), (3, 0.049360291004180906)]
INFO flwr 2023-05-02 08:06:12,316 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.098), (1, 0.302), (2, 0.388), (3, 0.432)]}


(launch_and_evaluate pid=1588558) [Client 7] evaluate, config: {}


In [20]:
# print("")
list(histFedAvg.metrics_centralized.values())

# print("")

[[(0, 0.098), (1, 0.302), (2, 0.388), (3, 0.432)]]

In [23]:
import pandas as pd
df_result = pd.DataFrame()
metrics_cen = list(histFedAvg.metrics_centralized.keys())
for metric in metrics_cen:
    df_result[f"c_{metric}"] = [h[1] for h in histFedAvg.metrics_centralized[metric][1:]]

metrics_dis = list(histFedAvg.metrics_distributed.keys())
for metric in metrics_dis:
    df_result[f"d_{metric}"] = [h[1] for h in histFedAvg.metrics_distributed[metric]]

df_result

,c_accuracy,d_accuracy
0,0.302,0.307333
1,0.388,0.404667
2,0.432,0.472667


As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [13]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [14]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-05-02 07:59:21,237 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-05-02 07:59:25,327	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-02 07:59:25,834 | app.py:180 | Flower VCE: Ray initialized with resources: {'accelerator_type:RTX': 1.0, 'object_store_memory': 36427366809.0, 'CPU': 16.0, 'GPU': 1.0, 'node:172.17.0.2': 1.0, 'memory': 74997189223.0}
INFO flwr 2023-05-02 07:59:25,837 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-02 07:59:25,837 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-02 07:59:25,837 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-02 07:59:25,838 | server.py:101 | FL starting
DEBUG flwr 2023-05-02 07:59:25,838 | server.py:218 | fit_round 1: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=1567097) [Client 484, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1567097) Epoch 1: train loss 0.10286653786897659, accuracy 0.022222222222222223
(launch_and_fit pid=1567097) Epoch 2: train loss 0.10202530026435852, accuracy 0.24444444444444444
(launch_and_fit pid=1567097) Epoch 3: train loss 0.1010870411992073, accuracy 0.2
(launch_and_fit pid=1567181) [Client 714, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1567181) Epoch 1: train loss 0.1024223044514656, accuracy 0.1111111111111111
(launch_and_fit pid=1567181) Epoch 2: train loss 0.1011638417840004, accuracy 0.2222222222222222
(launch_and_fit pid=1567181) Epoch 3: train loss 0.10011611878871918, accuracy 0.2
(launch_and_fit pid=1567243) [Client 726, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1567243) Epoch 1: train loss 0.10163050889968872, accuracy 0.08888888888888889
(launch_and_fit pid=1567243) Epoch

DEBUG flwr 2023-05-02 08:00:03,505 | server.py:232 | fit_round 1 received 25 results and 0 failures
WARNING flwr 2023-05-02 08:00:03,534 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-05-02 08:00:03,535 | server.py:168 | evaluate_round 1: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=1568622) [Client 739, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=1568622) Epoch 1: train loss 0.10255886614322662, accuracy 0.08888888888888889
(launch_and_fit pid=1568622) Epoch 2: train loss 0.10162127763032913, accuracy 0.17777777777777778
(launch_and_fit pid=1568622) Epoch 3: train loss 0.10058248788118362, accuracy 0.2222222222222222
(launch_and_evaluate pid=1568695) [Client 342] evaluate, config: {}
(launch_and_evaluate pid=1568758) [Client 921] evaluate, config: {}
(launch_and_evaluate pid=1568816) [Client 419] evaluate, config: {}
(launch_and_evaluate pid=1568874) [Client 445] evaluate, config: {}
(launch_and_evaluate pid=1568932) [Client 752] evaluate, config: {}
(launch_and_evaluate pid=1568993) [Client 577] evaluate, config: {}
(launch_and_evaluate pid=1569051) [Client 153] evaluate, config: {}
(launch_and_evaluate pid=1569109) [Client 645] evaluate, config: {}
(launch_and_evaluate pid=1569166) [Client 618] evaluate, 

DEBUG flwr 2023-05-02 08:01:11,099 | server.py:182 | evaluate_round 1 received 50 results and 0 failures
WARNING flwr 2023-05-02 08:01:11,100 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-05-02 08:01:11,100 | server.py:218 | fit_round 2: strategy sampled 25 clients (out of 1000)


(launch_and_evaluate pid=1571544) [Client 959] evaluate, config: {}
(launch_and_fit pid=1571616) [Client 981, round 2] fit, config: {'server_round': 2, 'local_epochs': 3}
(launch_and_fit pid=1571616) Epoch 1: train loss 0.10220684856176376, accuracy 0.08888888888888889
(launch_and_fit pid=1571616) Epoch 2: train loss 0.10075175017118454, accuracy 0.26666666666666666
(launch_and_fit pid=1571616) Epoch 3: train loss 0.09919605404138565, accuracy 0.28888888888888886
(launch_and_fit pid=1571678) [Client 33, round 2] fit, config: {'server_round': 2, 'local_epochs': 3}
(launch_and_fit pid=1571678) Epoch 1: train loss 0.10285479575395584, accuracy 0.13333333333333333
(launch_and_fit pid=1571678) Epoch 2: train loss 0.10152085870504379, accuracy 0.24444444444444444
(launch_and_fit pid=1571678) Epoch 3: train loss 0.10007794946432114, accuracy 0.24444444444444444
(launch_and_fit pid=1571737) [Client 324, round 2] fit, config: {'server_round': 2, 'local_epochs': 3}
(launch_and_fit pid=1571737) E

DEBUG flwr 2023-05-02 08:01:47,547 | server.py:232 | fit_round 2 received 25 results and 0 failures
DEBUG flwr 2023-05-02 08:01:47,574 | server.py:168 | evaluate_round 2: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=1573064) [Client 617, round 2] fit, config: {'server_round': 2, 'local_epochs': 3}
(launch_and_fit pid=1573064) Epoch 1: train loss 0.10162679851055145, accuracy 0.2222222222222222
(launch_and_fit pid=1573064) Epoch 2: train loss 0.10014112293720245, accuracy 0.2222222222222222
(launch_and_fit pid=1573064) Epoch 3: train loss 0.09941364079713821, accuracy 0.2222222222222222
(launch_and_evaluate pid=1573141) [Client 793] evaluate, config: {}
(launch_and_evaluate pid=1573199) [Client 816] evaluate, config: {}
(launch_and_evaluate pid=1573256) [Client 13] evaluate, config: {}
(launch_and_evaluate pid=1573314) [Client 449] evaluate, config: {}
(launch_and_evaluate pid=1573377) [Client 12] evaluate, config: {}
(launch_and_evaluate pid=1573434) [Client 439] evaluate, config: {}
(launch_and_evaluate pid=1573492) [Client 397] evaluate, config: {}
(launch_and_evaluate pid=1573551) [Client 609] evaluate, config: {}
(launch_and_evaluate pid=1573613) [Client 349] evaluate, conf

KeyboardInterrupt: 

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3 [WIP]](https://flower.dev/docs/tutorial/Flower-3-Building-a-Strategy-PyTorch.html) shows how to build a fully custom `Strategy` from scratch.